# Soccer Betting Model - Testing Notebook

This notebook is for testing the soccer betting model pipeline:
1. Building prompts
2. Evaluating bets
3. Analyzing results

In [ ]:
import pandas as pd
import datetime
from pathlib import Path
import sys

# Add scripts directory to path
sys.path.insert(0, '../scripts')

# Import our custom functions
from soccer_build_prompt import build_soccer_prompt, MODEL_LIST
from soccer_evaluate_bets import evaluate_soccer_bets

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 500)

## 1. Build Prompts for All Models

This will:
- Fetch upcoming soccer games
- Load historical performance
- Generate prompts for each model

In [ ]:
# Build prompts for all models
print("Building prompts for all soccer models...\n")

for model_name in MODEL_LIST:
    print(f"\n{'='*60}")
    print(f"Building prompt for: {model_name}")
    print(f"{'='*60}")
    
    try:
        df_games = build_soccer_prompt(model_name)
        print(f"\n✓ Successfully built prompt for {model_name}")
        print(f"  Games included: {len(df_games)}")
    except Exception as e:
        print(f"\n✗ Error building prompt for {model_name}: {e}")

print("\n" + "="*60)
print("Prompt building complete!")
print("="*60)

## 2. Test Single Model

Test with one model first to verify everything works

In [ ]:
# Test with charlie model
test_model = 'charlie'

print(f"Testing with model: {test_model}\n")
df_test = build_soccer_prompt(test_model)

print(f"\nGames DataFrame shape: {df_test.shape}")
print(f"\nFirst few games:")
df_test.head()

## 3. View Generated Prompt

Check the generated prompt file

In [ ]:
# Read and display the generated prompt
prompt_path = Path(f'../prompts/soccer_prompt_{test_model}.txt')

if prompt_path.exists():
    with open(prompt_path, 'r') as f:
        prompt_content = f.read()
    
    print(f"Prompt file: {prompt_path}")
    print(f"Length: {len(prompt_content):,} characters")
    print(f"\nFirst 1000 characters:")
    print("-" * 60)
    print(prompt_content[:1000])
    print("-" * 60)
else:
    print(f"Prompt file not found: {prompt_path}")

## 4. Check Existing Evaluated Bets

Load and analyze existing evaluated bets

In [ ]:
# Load existing evaluated bets
eval_path = Path('../data/soccer_bet_picks_evaluated.csv')

if eval_path.exists():
    df_eval = pd.read_csv(eval_path)
    print(f"Loaded {len(df_eval)} evaluated bets")
    print(f"\nModels: {df_eval['model'].unique().tolist()}")
    print(f"Date range: {df_eval['date'].min()} to {df_eval['date'].max()}")
    
    # Summary by model
    print(f"\n{'='*60}")
    print("Summary by Model")
    print(f"{'='*60}")
    
    summary = df_eval.groupby('model').agg({
        'bet_result': ['count', lambda x: (x == 'win').sum()],
        'bet_payout': 'sum'
    }).round(2)
    
    summary.columns = ['Total Bets', 'Wins', 'Total Payout']
    summary['Win Rate %'] = (summary['Wins'] / summary['Total Bets'] * 100).round(1)
    
    print(summary)
    
    # Display recent bets
    print(f"\n{'='*60}")
    print("Recent Bets (Last 10)")
    print(f"{'='*60}")
    df_eval.tail(10)
else:
    print(f"No evaluated bets file found at: {eval_path}")
    print("This is normal if you haven't evaluated any bets yet.")

## 5. Evaluate Bets (if you have pending bets)

This will evaluate any pending bets that have completed

In [ ]:
# Check if we have pending bets to evaluate
pending_bets_path = Path(f'../data/soccer_bets_{test_model}.txt')

if pending_bets_path.exists():
    print(f"Found pending bets file: {pending_bets_path}")
    
    # Load and preview
    df_pending = pd.read_csv(pending_bets_path)
    print(f"\nPending bets: {len(df_pending)}")
    print(f"\nFirst few bets:")
    display(df_pending.head())
    
    # Evaluate
    print(f"\n{'='*60}")
    print(f"Evaluating bets for {test_model}...")
    print(f"{'='*60}")
    
    df_evaluated = evaluate_soccer_bets(test_model)
    
    if df_evaluated is not None:
        print(f"\n✓ Evaluation complete!")
        display(df_evaluated.tail(10))
else:
    print(f"No pending bets file found: {pending_bets_path}")
    print("You'll need to create this file with your model's picks first.")

## 6. Analyze Performance Over Time

Visualize performance trends

In [ ]:
import matplotlib.pyplot as plt

if eval_path.exists():
    df_eval = pd.read_csv(eval_path)
    df_eval['date'] = pd.to_datetime(df_eval['date'])
    
    # Calculate cumulative payout by model
    df_eval_sorted = df_eval.sort_values(['model', 'date'])
    df_eval_sorted['cumulative_payout'] = df_eval_sorted.groupby('model')['bet_payout'].cumsum()
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    for model in df_eval_sorted['model'].unique():
        model_data = df_eval_sorted[df_eval_sorted['model'] == model]
        ax.plot(model_data['date'], model_data['cumulative_payout'], 
                marker='o', label=model, linewidth=2)
    
    ax.axhline(y=0, color='black', linestyle='--', alpha=0.3)
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Cumulative Payout (units)', fontsize=12)
    ax.set_title('Soccer Betting Model Performance Over Time', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No evaluation data available for plotting")

## 7. Quick Reference - File Locations

Important file paths for the soccer betting pipeline

In [ ]:
print("Soccer Betting Model File Structure")
print("="*60)
print("\nPrompts:")
for model in MODEL_LIST:
    path = Path(f'../prompts/soccer_prompt_{model}.txt')
    status = "✓" if path.exists() else "✗"
    print(f"  {status} {path}")

print("\nPending Bets:")
for model in MODEL_LIST:
    path = Path(f'../data/soccer_bets_{model}.txt')
    status = "✓" if path.exists() else "✗"
    print(f"  {status} {path}")

print("\nEvaluated Bets:")
path = Path('../data/soccer_bet_picks_evaluated.csv')
status = "✓" if path.exists() else "✗"
print(f"  {status} {path}")

print("\nGame Results:")
path = Path('../data/soccer_game_results.csv')
status = "✓" if path.exists() else "✗"
print(f"  {status} {path}")

print("\nDatabase:")
path = Path('../data/soccer_bets_db.csv')
status = "✓" if path.exists() else "✗"
print(f"  {status} {path}")
print("="*60)